In [1]:
import re
import random
from collections import defaultdict
from itertools import chain
import pandas as pd
import codecs
import json
import os
from glob import glob

In [2]:
data_dir= "../data/events/"

In [73]:
df_ls=[]
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        filepath = os.path.join(data_dir, file)
        dataf = pd.read_csv(filepath)
        df_ls.append(dataf)

In [74]:
dataframes = pd.concat(df_ls)

In [78]:
columns = ['event', 'leading_event', 'date', 'category', 'entities']

In [79]:
df = dataframes[columns]

In [82]:
df = df.sort_values(by=["date"])

In [84]:
df.to_csv("../data/events/events.csv", index=False)

# Export events dataframes

In [9]:
df_ls = []
for file in glob("../data/output/events_dict_**.json"):
    date = file.split("/")[-1].replace(".json", "").split("_")[-1]
    with open(file) as reader:
        data = json.load(reader)
    df= pd.DataFrame.from_dict(data, orient="index")
    df_ls.append(df)

In [10]:
df_merge = pd.concat(df_ls)

In [15]:
df_merge.sort_values(by=["date", "cateogry", "main_event"],inplace=True)

In [17]:
df_merge.to_csv("../data/events/events_20220224-20220304.csv", index=False)

In [11]:
file = data_dir+"events_dict_2022-02-28.json"
with open(file) as reader:
    data = json.load(reader)

In [61]:
def event_df(data):
    
    eventls=[]
    leadings =[]
    dates = []
    categories =[]
    entities_ls =[]
    
    count=0
    for idx, event_dict in data.items():
        events=event_dict["events"]
        leading = event_dict["leading"]
        date=event_dict["date"]
        category = event_dict["category"]
        entities=event_dict["entities"]
        
        # event_entity_dict=defaultdict(list)
        
        for event in events:
            entities4event=[]
            
            categories.append(category)
            if len(entities)>0:
                for entity in entities:
                    if entity in event or event in entity:
                        entities4event.append(entity)
            count+=1
        
            if len(entities4event)>0:
                entities_ls.append(list(set(entities4event)))
            else:
                entities_ls.append([])
            
        leadings.append([leading for _ in range(len(events))])
        dates.append([date for _ in range(len(events))])
        eventls.append(events)
        
    events_ = list(chain.from_iterable(eventls))
    leadings_ = list(chain.from_iterable(leadings))
    dates_ = list(chain.from_iterable(dates))
    
    
    df= pd.DataFrame.from_dict({
        "event":events_,
        "leading_event":leadings_,
        "date":dates_,
        "category":categories,
        "entities":entities_ls
    })
    
    return df
    

In [3]:
test_file = "../data/output/events_dict_2022-03-2.json"
date = test_file.split("/")[-1].replace(".json", "").split("_")[-1]
print(date)
with open(test_file) as reader:
    data= json.load(reader)

2022-03-2


In [7]:
df= pd.DataFrame.from_dict(data, orient="index")

In [ ]:
df

In [ ]:
event_df(data)